In [138]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime
from sqlalchemy.orm import Session
import seaborn

In [139]:
#the csv file
csvfile = "./hawaii_measurements.csv"

In [140]:
df = pd.read_csv(csvfile, dtype = object)

In [141]:
df.columns = ["station", 'date', 'rainfall', 'temp']

In [142]:
new_csv = "hawaii_cleaned.csv"
df = df.dropna(axis = 0, how = 'any')
df.to_csv(new_csv, index = False)


In [143]:
new_df = pd.read_csv('hawaii_cleaned.csv')

In [144]:
engine = create_engine("sqlite:///hawaii_hw.sqlite")
conn = engine.connect()
Base = declarative_base()

In [145]:
class Measurement(Base):
    __tablename__ = 'hawaii_measurement'
    
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    date = Column(Text)
    rainfall = Column(Integer)
    temp = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
        


In [146]:
class Station(Base):
    __tablename__ = 'hawaii_station'
    
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [147]:
Base.metadata.create_all(engine)

In [148]:
measurement_data = new_df.to_dict(orient = 'records')

In [149]:
read_stations = pd.read_csv('hawaii_stations.csv')

In [150]:
stations_dict = read_stations.to_dict(orient = 'records')

In [151]:
metadata = MetaData(bind = engine)
metadata.reflect()

In [152]:
#to see the metadata of the the measurement class 
Measurement.__table__ 

Table('hawaii_measurement', MetaData(bind=None), Column('id', Integer(), table=<hawaii_measurement>, primary_key=True, nullable=False), Column('station', Text(), table=<hawaii_measurement>), Column('date', Text(), table=<hawaii_measurement>), Column('rainfall', Integer(), table=<hawaii_measurement>), Column('temp', Integer(), table=<hawaii_measurement>), schema=None)

In [153]:
# assigning the dictionaries, now turned into sql tables, to a variable
table = sqlalchemy.Table('hawaii_measurement', metadata, autoload= True)
stations_table = sqlalchemy.Table('hawaii_station', metadata, autoload = True)

In [154]:
conn.execute(table.insert(), measurement_data)

In [155]:
conn.execute(stations_table.insert(), stations_dict)

In [156]:
conn.execute("select * from hawaii_measurement LIMIT 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0, 64),
 (7, 'USC00519397', '2010-01-09', 0, 68),
 (8, 'USC00519397', '2010-01-10', 0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0, 61)]

In [157]:
conn.execute("select distinct station, count(temp) from hawaii_measurement GROUP BY station ORDER BY count(temp) DESC").fetchall()
#USC00519281 has the most observations (Waihee station)


[('USC00519281', 8316),
 ('USC00513117', 8088),
 ('USC00519397', 8055),
 ('USC00519523', 7716),
 ('USC00516128', 7452),
 ('USC00514830', 5811),
 ('USC00511918', 5796),
 ('USC00517948', 2049),
 ('USC00518838', 1026)]